In [ ]:
import argparse
import json
import os
import sys
import torch
import torch.nn as nn
import re
import math
from tqdm import tqdm
from transformers import (
    AutoModel,
    AutoModelForCausalLM,
    AutoTokenizer,
    OPTForCausalLM,
    BloomForCausalLM,    
)
from torch.utils.data import DataLoader
import logging

notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))

std = 0.012528747320175171

if project_root not in sys.path:
    sys.path.append(project_root)

import models
from models import get_model

class Config:
    def __init__(self, **entries):
        self.__dict__.update(entries)

device = torch.device('cuda:5')

comp_model_path = '/home/jgryu/Weight_compression/VQVAE_v2/checkpoint/nwc_ql/block_seq_ql_random__llama-3-8b-hf/block_seq_ql_random_col_16/lmbda1000_rdloss_ql_encdim512_M16_batch_size2048_total_iter1500000_lr0.0001_seed100/recent_model_loss_7.16303_bpp_5.97462_MSE_0.00115_total_iter_170000.pth.tar'
comp_model_path = '/home/jgryu/Weight_compression/VQVAE_v2/checkpoint/nwc_ql/block_seq_ql_random__meta-llama--Llama-2-7b-hf__adapt_4096.pt/lmbda300_rdloss_ql_size16_encdim512_M16_batch_size1024_total_iter200000_lr0.0001_seed100/recent_model_loss_5.70953_bpp_5.64828_MSE_0.00301_total_iter_200000.pth.tar'
config = os.path.join(os.path.dirname(comp_model_path), 'config.json')
with open(config, 'r', encoding='utf-8') as file:
    config = json.load(file)
config = Config(**config)

if not hasattr(config, "Q"):
        config.Q = 4

comp_model = get_model(config.architecture, config, scale=torch.zeros(1), shift=torch.zeros(1))      

ckpt = torch.load(comp_model_path)

if 'scale' in ckpt["state_dict"]:
    del ckpt["state_dict"]['scale']
if 'shift' in ckpt["state_dict"]:
    del ckpt["state_dict"]['shift']

comp_model.load_state_dict(ckpt["state_dict"])

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'models'

In [13]:
total_params = sum(p.numel() for p in comp_model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 564208


In [14]:
count = 0
for name, param in comp_model.named_parameters():
    if 'g_a' in name: continue
    print(f"{name}: {param.numel()}")
    count +=  param.numel()
    
count

quality_embedding.weight: 2048
g_s.weight_in.weight: 8192
g_s.weight_in.bias: 512
g_s.weight_stack.0.lin_1.0.weight: 262144
g_s.weight_stack.0.lin_1.0.bias: 512
g_s.weight_stack.0.lin_1.1.weight: 512
g_s.weight_stack.0.lin_1.1.bias: 512
g_s.out.weight: 8192
g_s.out.bias: 16
entropy_bottleneck._matrix0: 48
entropy_bottleneck._bias0: 48
entropy_bottleneck._factor0: 48
entropy_bottleneck._matrix1: 144
entropy_bottleneck._bias1: 48
entropy_bottleneck._factor1: 48
entropy_bottleneck._matrix2: 144
entropy_bottleneck._bias2: 48
entropy_bottleneck._factor2: 48
entropy_bottleneck._matrix3: 144
entropy_bottleneck._bias3: 48
entropy_bottleneck._factor3: 48
entropy_bottleneck._matrix4: 48
entropy_bottleneck._bias4: 16
entropy_bottleneck.quantiles: 48


283616

In [ ]:
import torch
from matmul_had import *

/home/jgryu/Weight_compression/VQVAE_v2/recon_lm/matmul_had.py:96: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("quip_lib::hadamard")


In [20]:
w = torch.randn(4096, 4096)
hw = matmul_hadU(w)
w_hat = matmul_hadU(hw)

In [21]:
w_hat

tensor([[ 2.3052,  1.2386, -0.7914,  ..., -0.5430,  0.1173, -1.2914],
        [-0.1574, -0.2344,  0.0640,  ..., -0.5559, -2.1109, -0.3195],
        [-0.0472, -0.0049,  0.5701,  ...,  1.6446,  0.1710, -0.3300],
        ...,
        [-0.0803, -0.7596,  0.2771,  ...,  0.9104,  0.7887, -0.3427],
        [ 3.2827, -0.7055, -0.2649,  ..., -0.0517,  1.6282,  0.9684],
        [ 1.3440, -1.3535, -0.8028,  ...,  1.2691, -0.0721, -1.1403]])

In [22]:
w

tensor([[ 2.3052,  1.2386, -0.7914,  ..., -0.5430,  0.1173, -1.2914],
        [-0.1574, -0.2344,  0.0640,  ..., -0.5559, -2.1109, -0.3195],
        [-0.0472, -0.0049,  0.5701,  ...,  1.6446,  0.1710, -0.3300],
        ...,
        [-0.0803, -0.7596,  0.2771,  ...,  0.9104,  0.7887, -0.3427],
        [ 3.2827, -0.7055, -0.2649,  ..., -0.0517,  1.6282,  0.9684],
        [ 1.3440, -1.3535, -0.8028,  ...,  1.2691, -0.0721, -1.1403]])

In [23]:
hw

tensor([[-0.0895, -0.0894,  0.8875,  ..., -1.4697,  1.1138, -1.0637],
        [-0.7803,  0.7130,  0.4208,  ...,  0.8103, -0.4406,  0.3364],
        [ 0.3479,  0.0641, -1.3461,  ..., -0.4162, -2.1352,  1.6657],
        ...,
        [ 0.4868,  0.6456,  0.1941,  ...,  1.6534, -0.8680,  1.3202],
        [-0.3154, -0.5766,  1.4982,  ..., -1.4923,  0.3776, -1.1650],
        [ 1.2925,  0.4151,  1.9718,  ...,  0.4050,  0.3518,  0.1145]])